### To Change to Uplow update these following
- /home/epinyoan/git/momask-codes/models/mask_transformer/transformer_trainer.py
- /home/epinyoan/git/momask-codes/models/vq/model.py
- /home/epinyoan/git/momask-codes/models/mask_transformer/transformer.py

### T2M-GPT Transformer change these:
from 2024-01-19-20-40-27_2_GPT_randId0-.5_weightBySample_t2mgptTrans_logEvry50_n_layers4
- /models/mask_transformer/tools.py (only one line: just don't compute loss here)
- /models/mask_transformer/transformer.py


In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
import os
import torch
import numpy as np

from torch.utils.data import DataLoader
from os.path import join as pjoin

from models.mask_transformer.transformer import MaskTransformer
from models.mask_transformer.transformer_trainer import MaskTransformerTrainer
from models.vq.model import RVQVAE

from options.train_option import TrainT2MOptions

from utils.plot_script import plot_3d_motion
from utils.motion_process import recover_from_ric
from utils.get_opt import get_opt
from utils.fixseed import fixseed
from utils.paramUtil import t2m_kinematic_chain, kit_kinematic_chain

from data.t2m_dataset import Text2MotionDataset
from motion_loaders.dataset_motion_loader import get_dataset_motion_loader
from models.t2m_eval_wrapper import EvaluatorModelWrapper

/home/epinyoan/miniconda3/envs/momask/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from options.train_option import TrainT2MOptions
parser = TrainT2MOptions()
opt = parser.parse(is_mock=True)
opt.name

------------ Options -------------
batch_size: 64
checkpoints_dir: ./log/t2m
cond_drop_prob: 0.1
dataset_name: t2m
dropout: 0.2
eval_every_e: 10
ff_size: 1024
force_mask: False
gamma: 0.1
gpu_id: -1
gumbel_sample: False
is_continue: False
is_train: True
latent_dim: 384
log_every: 50
lr: 0.0002
max_epoch: 500
max_motion_length: 196
milestones: [50000]
n_heads: 6
n_layers: 8
name: 2024-01-21-19-57-56_t2m_nlayer8_nhead6_ld384_ff1024_cdp0.1_rvq6ns
save_latest: 500
seed: 3407
share_weight: False
unit_length: 4
vq_name: rvq_nq1_dc512_nc512
warm_up_iter: 2000
-------------- End ----------------


'2024-01-21-19-57-56_t2m_nlayer8_nhead6_ld384_ff1024_cdp0.1_rvq6ns'

In [4]:
opt.data_root = './dataset/HumanML3D'
opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
opt.joints_num = 22
opt.max_motion_len = 55
dim_pose = 263
radius = 4
fps = 20
kinematic_chain = t2m_kinematic_chain
dataset_opt_path = './checkpoints/t2m/Comp_v6_KLD005/opt.txt'


In [5]:
opt.vq_name = 'rvq_nq6_dc512_nc512_noshare_qdp0.2'
vq_opt = get_opt(f'/home/epinyoan/git/momask-codes/checkpoints/t2m/{opt.vq_name}/opt.txt', 'cuda')
vq_model = RVQVAE(vq_opt,
                dim_pose,
                vq_opt.nb_code,
                vq_opt.code_dim,
                vq_opt.output_emb_width,
                vq_opt.down_t,
                vq_opt.stride_t,
                vq_opt.width,
                vq_opt.depth,
                vq_opt.dilation_growth_rate,
                vq_opt.vq_act,
                vq_opt.vq_norm)
ckpt = torch.load(pjoin(vq_opt.checkpoints_dir, vq_opt.dataset_name, vq_opt.name, 'model', 'net_best_fid.tar'),
                        map_location='cpu')
model_key = 'vq_model' if 'vq_model' in ckpt else 'net'
vq_model.load_state_dict(ckpt[model_key])
print(f'Loading VQ Model {opt.vq_name}')

Reading /home/epinyoan/git/momask-codes/checkpoints/t2m/rvq_nq6_dc512_nc512_noshare_qdp0.2/opt.txt
Loading VQ Model rvq_nq6_dc512_nc512_noshare_qdp0.2


In [6]:
opt.log_dir = pjoin('./log/t2m/', opt.dataset_name, opt.name)
opt.device = 'cuda' #torch.device("cpu" if opt.gpu_id == -1 else "cuda:" + str(opt.gpu_id))
opt.text_dir = pjoin(opt.data_root, 'texts')
opt.num_tokens = vq_opt.nb_code
clip_version = 'ViT-B/32'
t2m_transformer = MaskTransformer(code_dim=vq_opt.code_dim,
                                      cond_mode='text',
                                      latent_dim=opt.latent_dim,
                                      ff_size=opt.ff_size,
                                      num_layers=opt.n_layers,
                                      num_heads=opt.n_heads,
                                      dropout=opt.dropout,
                                      clip_dim=512,
                                      cond_drop_prob=opt.cond_drop_prob,
                                      clip_version=clip_version,
                                      opt=opt)
# t2m_transformer.load_and_freeze_token_emb(vq_model.quantizer_upper.codebooks[0], vq_model.quantizer_lower.codebooks[1])

latent_dim: 384, ff_size: 1024, nlayers: 8, nheads: 6, dropout: 0.2
Loading CLIP...


In [21]:
# model_dir = '/home/epinyoan/git/momask-codes/log/t2m/t2m/2024-01-18-23-27-31_2_GPT_randId0-.5_weightBySample/model/latest.tar'
model_dir = '/home/epinyoan/git/momask-codes/log/t2m/t2m/2024-01-21-09-10-30_3_GPT_randId0-.5_weightBySample_FIXED_TRAIN_END_TOKEN_b256/model/latest.tar'
checkpoint = torch.load(model_dir, map_location='cuda')
missing_keys, unexpected_keys = t2m_transformer.load_state_dict(checkpoint['t2m_transformer'], strict=False)

In [8]:
mean = np.load(pjoin('./checkpoints/', opt.dataset_name, opt.vq_name, 'meta', 'mean.npy'))
std = np.load(pjoin('./checkpoints/', opt.dataset_name, opt.vq_name, 'meta', 'std.npy'))

train_split_file = pjoin(opt.data_root, 'train.txt')
val_split_file = pjoin(opt.data_root, 'val.txt')

train_dataset = Text2MotionDataset(opt, mean, std, train_split_file)
val_dataset = Text2MotionDataset(opt, mean, std, val_split_file)

train_loader = DataLoader(train_dataset, batch_size=opt.batch_size, num_workers=4, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=opt.batch_size, num_workers=4, shuffle=True, drop_last=True)

eval_val_loader, _ = get_dataset_motion_loader(dataset_opt_path, 32, 'val', device=opt.device)

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)


  1%|          | 222/23384 [00:00<00:10, 2212.81it/s]

100%|██████████| 1460/1460 [00:00<00:00, 1862.21it/s]


Reading ./checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading dataset t2m ...


100%|██████████| 1460/1460 [00:00<00:00, 1890.54it/s]


Pointer Pointing at 0
Ground Truth Dataset Loading Completed!!!
Reading ./checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


In [9]:
opt.is_train = False # to turn of tensorboard
trainer = MaskTransformerTrainer(opt, t2m_transformer, vq_model)
# trainer.resume(model_dir = '/home/epinyoan/git/momask-codes/log/t2m/t2m/2024-01-13-22-25-36_1_uplow_vq512emb_mileStone100k_800ep/model/latest.tar')
# vq_model.moment = {'mean': torch.from_numpy(train_loader.dataset.mean).cuda().float(), 
#                 'std': torch.from_numpy(train_loader.dataset.std).cuda().float()}
t2m_transformer.to(opt.device)
vq_model.to(opt.device)
''

''

In [19]:
batch_data = next(iter(train_loader))
conds, motion, m_lens = batch_data
motion = motion.detach().float().to(opt.device)
m_lens = m_lens.detach().long().to(opt.device)

# (b, n, q)
code_idx, _ = vq_model.encode(motion)
m_lens = m_lens // 4

conds = conds.to(opt.device).float() if torch.is_tensor(conds) else conds

_loss, _pred_ids, _acc = t2m_transformer(code_idx[..., 0], conds, m_lens)
_loss, _pred_ids.shape, _acc

x_ids: torch.Size([64, 49])


(tensor(1.8699, device='cuda:0', grad_fn=<SumBackward0>),
 torch.Size([64, 50]),
 0.4661746621131897)

In [11]:
# Test "pad_when_end"
xs = torch.tensor([[1,2,3,4,5,6],
                   [1,2,3,4,512,512],
                   [1,512,3,4,5,512],
                   [512,2,3,4,5,6],], device='cuda')
t2m_transformer.pad_when_end(xs)

(tensor([[ 1,  2,  3,  4,  5,  6],
         [ 1,  2,  3,  4, -1, -1],
         [ 1, -1, -1, -1, -1, -1],
         [ 0, -1, -1, -1, -1, -1]], device='cuda:0'),
 tensor([7, 4, 1, 1], device='cuda:0'))

In [12]:
word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token = next(iter(eval_val_loader))
m_length = m_length.cuda()
time_steps = 18
cond_scale = 4
mids = t2m_transformer.generate(clip_text, m_length//4, time_steps, cond_scale, temperature=1)
mids, pred_len = t2m_transformer.pad_when_end(mids)
mids.unsqueeze_(-1)
pred_motions = vq_model.forward_decoder(mids)

In [13]:
mids.shape

torch.Size([32, 49, 1])

### Eval

In [23]:
from utils.eval_t2m import evaluation_mask_transformer
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
best_fid, best_div, best_top1, best_top2, best_top3, best_matching, writer = evaluation_mask_transformer(
        'opt.save_root', eval_val_loader, t2m_transformer, vq_model, logger, 0,
        best_fid=100, best_div=100,
        best_top1=0, best_top2=0, best_top3=0,
        best_matching=100, eval_wrapper=eval_wrapper,
        plot_func=lambda x:x, save_ckpt=False, save_anim=False
    )

--> 	 Eva. Ep 0 :, FID. 0.1432, Diversity Real. 9.9017, Diversity. 9.6550, R_precision_real. [0.51795213 0.69481383 0.79654255], R_precision. [0.50864362 0.6974734  0.79654255], matching_score_real. 2.9001340003723795, matching_score_pred. 2.9835535262493376
./Test/FID 0.1432133627930483 0
./Test/Diversity 9.655023 0
./Test/top1 0.5086436170212766 0
./Test/top2 0.6974734042553191 0
./Test/top3 0.7965425531914894 0
./Test/matching_score 2.9835535262493376 0
--> --> 	 FID Improved from 100.00000 to 0.14321 !!!
--> --> 	 matching_score Improved from 100.00000 to 2.98355 !!!
--> --> 	 Diversity Improved from 100.00000 to 9.65502 !!!
--> --> 	 Top1 Improved from 0.0000 to 0.5086 !!!
--> --> 	 Top2 Improved from 0.0000 to 0.6975 !!!
--> --> 	 Top3 Improved from 0.0000 to 0.7965 !!!
